# Verify Waymo Data

In [1]:
!ls waymo_data_v120/training

0-of-10.tfrecord  3-of-10.tfrecord  6-of-10.tfrecord  9-of-10.tfrecord
1-of-10.tfrecord  4-of-10.tfrecord  7-of-10.tfrecord
2-of-10.tfrecord  5-of-10.tfrecord  8-of-10.tfrecord


In [2]:
!ls waymo_data_v120/validation

0-of-10.tfrecord  3-of-10.tfrecord  6-of-10.tfrecord  9-of-10.tfrecord
1-of-10.tfrecord  4-of-10.tfrecord  7-of-10.tfrecord
2-of-10.tfrecord  5-of-10.tfrecord  8-of-10.tfrecord


# Configure Model/Parameters

In [3]:
MODELS_CONFIG = {
    'efficientdet_d4': {
        'model_name': 'efficientdet_d4_coco17_tpu-32',
        'base_pipeline_file': 'efficientdet_d4_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d4_coco17_tpu-32.tar.gz',
        'batch_size':2
    }
}

chosen_model = 'efficientdet_d4'
num_steps = 50000
num_eval_steps = 500

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size']
pipeline_fname = './'+ model_name + '/pipeline.config'
fine_tune_checkpoint = './' + model_name + '/checkpoint/ckpt-0'
label_map_pbtxt_fname = 'waymo_labelmap.txt'
train_record_fname = "./waymo_data_v120/training/*"
test_record_fname = "./waymo_data_v120/validation/*"

In [4]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)
num_classes

/home/xenowulf/anaconda3/envs/tf/lib/python3.7/site-packages/scipy/__init__.py:140: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.0)
  UserWarning)


3

In [5]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

writing custom configuration file


# Training

In [16]:
PIPELINE_CONFIG_PATH='./pipeline_file.config'
MODEL_DIR='./outputmodel_singlegpu_waymo_v120_efficientdet_d4'
!CUDA_VISIBLE_DEVICES=1 python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={PIPELINE_CONFIG_PATH} \
    --model_dir={MODEL_DIR} \
    --alsologtostderr

2021-04-13 09:20:27.184387: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/xenowulf/anaconda3/envs/tf/lib/python3.7/site-packages/scipy/__init__.py:140: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.0)
  UserWarning)
2021-04-13 09:20:28.475053: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-13 09:20:28.475702: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-13 09:20:28.574779: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2021-04-13 09:20:28.574814: I tensorflow/stream_executor/platform/default/dso_

# Evaluation

In [17]:
PIPELINE_CONFIG_PATH='./pipeline_file.config'
MODEL_DIR='./outputmodel_singlegpu_waymo_v120_efficientdet_d4'
CHECKPOINT_DIR='./outputmodel_singlegpu_waymo_v120_efficientdet_d4'
!CUDA_VISIBLE_DEVICES=1 python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={PIPELINE_CONFIG_PATH} \
    --model_dir={MODEL_DIR} \
    --checkpoint_dir={CHECKPOINT_DIR} \
    --alsologtostderr

2021-04-14 20:31:18.258526: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/xenowulf/anaconda3/envs/tf/lib/python3.7/site-packages/scipy/__init__.py:140: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.0)
  UserWarning)
2021-04-14 20:31:19.575432: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-14 20:31:19.576027: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-14 20:31:19.690806: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:02:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2021-04-14 20:31:19.690840: I tensorflow/stream_executor/platform/default/dso_

# Export Model

In [18]:
!python models/research/object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path {PIPELINE_CONFIG_PATH} \
    --trained_checkpoint_dir ./outputmodel_singlegpu_waymo_v120_efficientdet_d4 \
    --output_directory ./exportmodel_singlegpu_waymo_v120_efficientdet_d4

2021-04-15 09:28:15.899310: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/xenowulf/anaconda3/envs/tf/lib/python3.7/site-packages/scipy/__init__.py:140: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.16.0)
  UserWarning)
2021-04-15 09:28:17.590020: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-15 09:28:17.590651: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-15 09:28:17.843801: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2021-04-15 09:28:17.845488: I tensorflow/core/common_runtime/gpu/gpu_device.cc